# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [46]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Read the CSV into DataFrame

In [47]:
# Load the data
file_path = Path('lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [48]:
# Fitting and encoding the columns with the LabelEncoder
le = LabelEncoder()
# Encoding home_ownership column
le.fit(df["homeowner"])
df["homeowner"] = le.transform(df["homeowner"])


# Encoding loan_status column
le.fit(df["loan_status"])
df["loan_status"] = le.transform(df["loan_status"])



# Split the Data into Training and Testing

In [49]:
# Create our features
X = df.drop(columns="loan_status")
# Create our target
y =df["loan_status"]

In [50]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [51]:
# Check the balance of our target values
pd.DataFrame(df["loan_status"]).value_counts()

loan_status
1              75036
0               2500
dtype: int64

In [52]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=76)
X_train.shape

(58152, 8)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [53]:
# Create the StandardScaler instance

scaler = StandardScaler()


In [54]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
x_scaler = scaler.fit(X_train)

In [55]:
# Scale the training and testing data
X_train_scaled = x_scaler.transform(X_train)
X_test_scaled = x_scaler.transform(X_test)

In [73]:
def run_logistic_regression_model(x_train,y_train,x_test, y_test):
    model = LogisticRegression(solver='lbfgs', random_state=76)
    model.fit(x_train, y_train) 
    # Calculated the balanced accuracy score
    y_pred = model.predict(X_test)
    acc_score=balanced_accuracy_score(y_test, y_pred)
    # Display the confusion matrix
    cm_df= confusion_matrix(y_test, y_pred)
    # Displaying results
    print("Confusion Matrix")
    display(cm_df)
    print(f"Balanced Accuracy Score : {acc_score}")
    print("Classification Report")
    print(classification_report_imbalanced(y_test, y_pred))
    return

# Simple Logistic Regression

In [74]:
run_logistic_regression_model(X_train_scaled, y_train, X_test_scaled, y_test)

Confusion Matrix


array([[    0,   617],
       [    0, 18767]], dtype=int64)

Balanced Accuracy Score : 0.5
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      1.00      0.00      0.00      0.00       617
          1       0.97      1.00      0.00      0.98      0.00      0.00     18767

avg / total       0.94      0.97      0.03      0.95      0.00      0.00     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [75]:

# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
# View the count of target classes with Counter
Counter(y_resampled)

Counter({1: 56269, 0: 56269})

In [76]:
# Train the Logistic Regression model using the resampled data
run_logistic_regression_model(X_resampled, y_resampled, X_test_scaled,y_test)


Confusion Matrix


array([[  617,     0],
       [18767,     0]], dtype=int64)

Balanced Accuracy Score : 0.5
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      1.00      0.00      0.06      0.00      0.00       617
          1       0.00      0.00      1.00      0.00      0.00      0.00     18767

avg / total       0.00      0.03      0.97      0.00      0.00      0.00     19384



### SMOTE Oversampling

In [77]:
# Resample the training data with SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)
# View the count of target classes with Counter
Counter(y_resampled)

Counter({1: 56269, 0: 56269})

In [78]:
# Train the Logistic Regression model using the resampled data

run_logistic_regression_model(X_resampled, y_resampled, X_test_scaled,y_test)


Confusion Matrix


array([[  617,     0],
       [18767,     0]], dtype=int64)

Balanced Accuracy Score : 0.5
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      1.00      0.00      0.06      0.00      0.00       617
          1       0.00      0.00      1.00      0.00      0.00      0.00     18767

avg / total       0.00      0.03      0.97      0.00      0.00      0.00     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [79]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({0: 1883, 1: 1883})

In [80]:
# Train the Logistic Regression model using the resampled data

run_logistic_regression_model(X_resampled, y_resampled, X_test_scaled,y_test)

Confusion Matrix


array([[    0,   617],
       [    0, 18767]], dtype=int64)

Balanced Accuracy Score : 0.5
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      1.00      0.00      0.00      0.00       617
          1       0.97      1.00      0.00      0.98      0.00      0.00     18767

avg / total       0.94      0.97      0.03      0.95      0.00      0.00     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [81]:
# Resample the training data with SMOTEENN
sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({0: 55529, 1: 55934})

In [82]:
run_logistic_regression_model(X_resampled, y_resampled, X_test_scaled,y_test)

Confusion Matrix


array([[  617,     0],
       [18767,     0]], dtype=int64)

Balanced Accuracy Score : 0.5
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      1.00      0.00      0.06      0.00      0.00       617
          1       0.00      0.00      1.00      0.00      0.00      0.00     18767

avg / total       0.00      0.03      0.97      0.00      0.00      0.00     19384



# Final Questions

1. Which model had the best balanced accuracy score?

    They all have the same score
2. Which model had the best recall score?

    The simple one, the under-sampling and the combined

3. Which model had the best geometric mean score?

    They all have zero.
